################################################################################################################################################
3. ID  : 
################################################################################################################################################

################################################################################################################################################

################################################################################################################################################

################################################################################################################################################

################################################################################################################################################
4. ID 2136 : Customer Tracking
################################################################################################################################################
Given the users' sessions logs on a particular day, calculate how many hours each user was active that day.
Note: The session starts when state=1 and ends when state=0.
################################################################################################################################################
Logic:
Each time we see state = 1, the user starts a session.
Each time we see state = 0, that session ends.
We pair each start with the next end for the same user.
Then we calculate the difference between end time and start time.
Sum up the durations for each user for that day (convert to hours).
################################################################################################################################################
SELECT
  t.cust_id,  -- we want one result per customer
  ROUND(SUM(TIMESTAMPDIFF(MINUTE, t.ts_start, t.ts_end)) / 60.0, 1) AS hour_diff
  --  TIMESTAMPDIFF gives minutes between start & end
  --  Divide by 60 to get hours
  --   Round to 1 decimal to match answers like 4.5
FROM (
  --  Step A: pair each start (state=1) with its next end (state=0)
  SELECT
    s.cust_id,
    s.timestamp AS ts_start,
    (
      SELECT MIN(e.timestamp)
      FROM cust_tracking e
      WHERE e.cust_id = s.cust_id
        AND e.state = 0            -- find an end time
        AND e.timestamp > s.timestamp  -- it happens after the start
    ) AS ts_end
  FROM cust_tracking s
  WHERE s.state = 1                -- only look at session starts
) AS t
WHERE t.ts_end IS NOT NULL         -- skip starts that never ended
GROUP BY t.cust_id
ORDER BY t.cust_id;

-- WITH CTE AS (
--    SELECT cust_id,
--          state,
--           timestamp,
--           LEAD(timestamp) OVER (PARTITION BY cust_id ORDER BY timestamp ASC) AS stop
--    FROM cust_tracking
-- )
-- SELECT cust_id,
--       SUM(TIMESTAMPDIFF(MINUTE, timestamp, stop)/60)
-- FROM CTE    
-- WHERE state = 1
-- GROUP BY 1
############################################################################################################################################

################################################################################################################################################
3. ID 2089 : Cookbook Recipes
################################################################################################################################################
number generation, self-joins, and conditional logic
################################################################################################################################################
You are given a table containing recipe titles and their corresponding page numbers from a cookbook. Your task is to format the data to represent how recipes are distributed across double-page spreads in the book.


Each spread consists of two pages:


⦁   The left page (even-numbered) and its corresponding recipe title (if any).
⦁   The right page (odd-numbered) and its corresponding recipe title (if any).


The output table should contain the following three columns:


⦁   left_page_number – The even-numbered page that starts each double-page spread.
⦁   left_title – The title of the recipe on the left page (if available).
⦁   right_title – The title of the recipe on the right page (if available).


For the  k-th  row (starting from 0):


⦁   The  left_page_number  should be 2 × k.
⦁   The  left_title  should be the title from page 2 × k, or NULL if there is no recipe on that page.
⦁   The  right_title  should be the title from page 2 × k + 1, or NULL if there is no recipe on that page.


Each page contains at most one recipe and  if a page does not contain a recipe, the corresponding title should be NULL. Page 0 (the inside cover) is always empty and included in the output. Only include spreads where at least one of the two pages has a recipe.
################################################################################################################################################
Each double-page spread has:
A left page (even-numbered, like 0, 2, 4…)
A right page (the next one, odd-numbered, like 1, 3, 5…)
We want:
left_page_number	left_title	right_title
0	NULL	Pancakes
2	Omelette	French Toast
6	Brownies	NULL
🧠 Approach
Every spread starts on even-numbered pages (0, 2, 4, …).
We’ll self-join the table:
One copy for left pages (even pages)
One copy for right pages (odd pages = left page + 1)
Include page 0 (inside cover), even if there’s no recipe.
################################################################################################################################################
################################################################################################################################################
################################################################################################################################################

In [ ]:
################################################################################################################################################
2. ID 2059 : Player with Longest Streak
################################################################################################################################################
You are given a table of tennis players and their matches that they could either win (W) or lose (L). Find the longest streak of wins. A streak is a set of consecutive won matches of one player. The streak ends once a player loses their next match. Output the ID of the player or players and the length of the streak.
################################################################################################################################################
TABLE       : players_results
match_date  : date
match_result: text
player_id   : bigint
################################################################################################################################################
| player_id | match_date | match_result |
| --------- | ---------- | ------------ |
| 1 (Alex)  | Jan 1      | W            |
| 1         | Jan 2      | W            |
| 1         | Jan 3      | L            |
| 1         | Jan 4      | W            |
| 1         | Jan 5      | W            |
| 1         | Jan 6      | W            |
| 2 (Ben)   | Jan 1      | L            |
| 2         | Jan 2      | W            |
| 2         | Jan 3      | W            |
| 2         | Jan 4      | L            |
##############################################################################################################################################
step 1. count how many W’s happen one after another — and stop counting when there’s an L (loss).
----------------------------------------------------------------------------------------------
Date	Result	Streak Count	Why for Alex
---------------------------------------------------------------------------------------------
Jan 1	W	    1	            (start streak)
Jan 2	W	    2	            (still winning)
Jan 3	L	    —	            (streak breaks)
Jan 4	W	    1	            (new streak starts)
Jan 5	W	    2	            (still winning)
Jan 6	W	    3	            (longest streak)
------------------------------------------------------------------------------------------------------
step 2 : Count how many losses happened so far : Keep counting losses for each player, in date order.
The “loss counter” goes up each time they lose. That number becomes a group number for each streak.
SUM(CASE WHEN match_result = 'L' THEN 1 ELSE 0 END)
OVER (PARTITION BY player_id ORDER BY match_date)
------------------------------------------
Date	Result	LossCounter
-------------------------------------------
Jan 1	W	    0
Jan 2	W	    0
Jan 3	L	    1
Jan 4	W	    1
-------------------------------------------

Step 3. Group by (player_id + loss counter)
When we group by that “loss counter", all wins between two losses stay together!
----------------------------------------
player_id	loss counter	# of wins
---------------------------------------
1	        0	                2
1	        1	                3
---------------------------------------

Step 4. — Find the longest number of wins per player. We take the maximum streak length for each player. Then we find the biggest of all (the global winner ).

Final result:
-----------------------------
player_id	longest_streak
------------------------------
1	            3
-----------------------------       
Alex wins! 
------------------------------

Summary:
------------------------------------------------------------------------
Step	What happens	            Like saying
-------------------------------------------------------------------------
1️.	    Count losses as we go	        “Every time you lose, start a new chapter.”
2️.  	Group wins between losses	    “Count wins in each chapter.”
3️.     Pick the biggest number	        “Who had the longest winning run?”
################################################################################################################################################

-- STEP 1: Create loss counter
-- Every time a player loses (L), the counter goes up.
-- This lets us "group" consecutive wins between losses.

SELECT 
    player_id,
    match_date,
    match_result,
    SUM(CASE WHEN match_result = 'L' THEN 1 ELSE 0 END)
        OVER (PARTITION BY player_id ORDER BY match_date) AS loss_group
FROM players_results
ORDER BY player_id, match_date
LIMIT 10

-- Example Output:
-- player_id | match_date  | match_result | loss_group
-- -----------+-------------+--------------+------------
-- 1          | 2024-01-01  | W            | 0
-- 1          | 2024-01-02  | W            | 0
-- 1          | 2024-01-03  | L            | 1
-- 1          | 2024-01-04  | W            | 1
-- 1          | 2024-01-05  | W            | 1
-- 2          | 2024-01-01  | W            | 0
-- 2          | 2024-01-02  | L            | 1
-- 2          | 2024-01-03  | W            | 1



-- STEP 2️: Group wins between losses
-- Now count how many consecutive wins happen per player per "loss group"

WITH loss_counter AS (
    SELECT 
        player_id,
        match_date,
        match_result,
        SUM(CASE WHEN match_result = 'L' THEN 1 ELSE 0 END)
            OVER (PARTITION BY player_id ORDER BY match_date) AS loss_group
    FROM players_results
)
SELECT 
    player_id,
    loss_group,
    COUNT(*) AS win_streak
FROM loss_counter
WHERE match_result = 'W'
GROUP BY player_id, loss_group
ORDER BY player_id, loss_group
LIMIT 10

-- Example Output:
-- player_id | loss_group | win_streak
-- -----------+-------------+------------
-- 1          | 0           | 2
-- 1          | 1           | 2
-- 2          | 0           | 1
-- 2          | 1           | 1



-- STEP 3️: Find longest streak per player
-- Pick the maximum streak for each player

WITH loss_counter AS (
    SELECT 
        player_id,
        match_date,
        match_result,
        SUM(CASE WHEN match_result = 'L' THEN 1 ELSE 0 END)
            OVER (PARTITION BY player_id ORDER BY match_date) AS loss_group
    FROM players_results
),
win_streaks AS (
    SELECT 
        player_id,
        loss_group,
        COUNT(*) AS win_streak
    FROM loss_counter
    WHERE match_result = 'W'
    GROUP BY player_id, loss_group
)
SELECT 
    player_id,
    MAX(win_streak) AS max_streak
FROM win_streaks
GROUP BY player_id
ORDER BY player_id
LIMIT 10

-- Example Output:
-- player_id | max_streak
-- -----------+------------
-- 1          | 2
-- 2          | 1



-- STEP 4️.: Find global winner(s)
-- Compare each player's best streak with the overall max streak.

WITH loss_counter AS (
    SELECT 
        player_id,
        match_date,
        match_result,
        SUM(CASE WHEN match_result = 'L' THEN 1 ELSE 0 END)
            OVER (PARTITION BY player_id ORDER BY match_date) AS loss_group
    FROM players_results
),
win_streaks AS (
    SELECT 
        player_id,
        loss_group,
        COUNT(*) AS win_streak
    FROM loss_counter
    WHERE match_result = 'W'
    GROUP BY player_id, loss_group
),
max_win_streaks AS (
    SELECT 
        player_id,
        MAX(win_streak) AS max_streak
    FROM win_streaks
    GROUP BY player_id
)
SELECT 
    player_id,
    max_streak AS longest_win_streak
FROM max_win_streaks
WHERE max_streak = (SELECT MAX(max_streak) FROM max_win_streaks)
ORDER BY player_id

-- Example Output:
-- player_id | longest_win_streak
-- -----------+-------------------
-- 1          | 2

################################################################################################################################################

################################################################################################################################################
1. ID514: Marketing Campaign Success [Advanced]
################################################################################################################################################
You have the marketing_campaign table, which records in-app purchases by users. Users making their first in-app purchase enter a marketing campaign, where they see call-to-actions for more purchases. Find how many users made additional purchases due to the campaign's success.

The campaign starts one day after the first purchase. Users with only one or multiple purchases on the first day do not count, nor do users who later buy only the same products from their first day.

TABLE       : marketing_campaign
created_at  : date
price       : bigint
product_id  : bigint
quantity    : bigint
user_id     : bigint
##################################################################################################################################################

Step-by-Step Logic: 
Step 1️. — Find each user’s first purchase date - MIN(created_at)
Step 2️. — Find what products they bought on that first day
Step 3️. — Find purchases after first day and check if: date is later than first purchase day + product is different from their first-day products.
Step 4️. — Count users who made such new purchases
##################################################################################################################################################

WITH 

first_day AS (
    SELECT user_id, MIN(created_at) AS first_purchase_date
    FROM marketing_campaign
    GROUP BY user_id
),

first_purchase_products AS (
    SELECT m.user_id, m.product_id
    FROM marketing_campaign m
    JOIN first_day f 
      ON f.user_id = m.user_id 
     AND f.first_purchase_date = m.created_at
),

later_purchases AS (
    SELECT m.user_id, m.product_id
    FROM marketing_campaign m
    JOIN first_day f 
      ON f.user_id = m.user_id 
     AND m.created_at > f.first_purchase_date
)

SELECT COUNT(DISTINCT lp.user_id) AS campaign_success_users
FROM later_purchases lp
LEFT JOIN first_purchase_products fp 
  ON lp.user_id = fp.user_id 
 AND lp.product_id = fp.product_id
WHERE fp.product_id IS NULL-- means user bought a different product
################################################################################################################################################